In [1]:
import csv
import os, random
import glob
from pydub import AudioSegment

C:\Users\koool\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
######################[CAN BE UPDATED]###################
dialect = "37_degrees_series" 
# The root folder where the dialect is exist
dataset_path = r"D:/02_batch_2/{}/".format(dialect)
wanted_duration = 0.5 #in hours
number_of_dir = 10 #total number of files you want to separate the segments to 
########################################################
print(dataset_path)

D:/02_batch_2/37_degrees_series/


In [3]:
######################[DO NOT CHANGE]###################
files = []
segments_path = dataset_path + "segments/"
dir_counter = 0
for root, dirs, files in os.walk(segments_path, topdown=False):
    dir_counter = len(dirs) + 1
    break

current_path = segments_path +"/"+ str(dir_counter) + "/"
total_duration = 0
current_duration = 0

if not os.path.exists(segments_path):
    os.mkdir(segments_path)

if not os.path.exists(current_path):
    os.mkdir(current_path)

# Create CSV
header = ['segment path', 'transcript']
f = open(segments_path + "segments.csv", "w", encoding="utf8")
writer = csv.writer(f)
writer.writerow(header)
########################################################

25

# Step 1: Open dir and Select random text file

In [4]:
def select_random_file():
    os.chdir(dataset_path)
    myFiles = glob.glob('*.txt')
    text_file = random.choice(myFiles) #change dir name to whatever
    audio_file = text_file.replace(".txt", ".wav")
    read_random_line(text_file, audio_file)

# Step 2: Read random line

In [5]:
def read_random_line(text_file, audio_file):
    global total_duration
    global current_duration

    try:
        lines = open(dataset_path + text_file, encoding="utf8").read().splitlines()
        random_line = random.choice(lines)

        columns = random_line.split("\t")
        start = float(columns[2])
        end = float(columns[3])
        transcript = columns[4]
        duration = end - start

        segment_name = audio_file.replace(".wav", "") + '_' + str(start)  + '_' + str(end) + ".wav"

        print(segment_name, duration)
        # if segment is not valid read another one
        if segment_name in files and duration < 1:
            read_random_line(text_file, audio_file)
        else:
            total_duration += duration 
            current_duration += duration
            split_audio(audio_file, start, end,transcript, segment_name)
    except ValueError:
        read_random_line(text_file, audio_file)

# Step 3: Split the audio into segment 

In [6]:

def split_audio(audio_file, start, end, transcript, segment_name):
    t1 = start * 1000 #Works in milliseconds
    t2 = end * 1000
    new_audio_path = current_path+segment_name
    new_audio = AudioSegment.from_wav(dataset_path + audio_file)
    new_audio = new_audio[t1:t2]
    new_audio.export(new_audio_path, format="wav") #Exports to a wav file in the current path.
    write_segment(new_audio_path, transcript)

# Step 4: Write the new segment and write in the CSV

In [7]:
def write_segment(segment_name, transcript):
    writer.writerow([segment_name, transcript])
    files.append(segment_name)

# MAIN

In [8]:
if __name__ == "__main__":
    while total_duration < wanted_duration * 60 * 60:
        if current_duration > wanted_duration* 60 * 60 / number_of_dir:
            dir_counter += 1
            current_path = segments_path +"/"+ str(dir_counter) + "/"
            current_duration = 0

            if not os.path.exists(current_path):
                os.mkdir(current_path)
        
        select_random_file()

37_degrees_series_1-3_12.148654153846_13.373518461538.wav 1.224864307692
37_degrees_series_14-0_180.58122317597_184.08037490533.wav 3.499151729359994
37_degrees_series_21-2_9.5243088286109_11.574160072564.wav 2.0498512439531
37_degrees_series_15-2_60.320820580744_61.07076861946.wav 0.7499480387159991
37_degrees_series_13-2_26.823293319611_29.398129491018.wav 2.574836171407
37_degrees_series_9-2_148.216601983_150.24148725212.wav 2.0248852691200057
37_degrees_series_6-4_7.0991200367225_9.6987977966491.wav 2.5996777599265997
37_degrees_series_12-7_260.0089088699_263.78367526153.wav 3.77476639163001
37_degrees_series_13-2_13.799122005988_16.348959767964.wav 2.5498377619760006
37_degrees_series_5-4_242.68674804992_245.5365924337.wav 2.8498443837800096
37_degrees_series_13-2_77.070096276198_77.720054921407.wav 0.6499586452089972
37_degrees_series_20-3_389.36326644668_391.4380707042.wav 2.074804257520043
37_degrees_series_12-12_107.76816644022_109.19307608696.wav 1.424909646740005
37_degrees_